In [20]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0).astype(str)
print(df.shape)
df.head(3)

(4239, 17)


,59,45A,46A,47A,31D,44C,48,71D,78,50,51A,LCNO,from,EXPNO,LCBK,CU1,SPEC
3,"FORMOSA PLASTICS CORPORATION201,TUNG HWA N ROA...","MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...",1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,210321TAIWAN,210228,21/(REFER FIELD 47A CLAUSE NO.8),ALL BANK CHARGES OUTSIDE INDIA ARETO THE ACCOU...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...,nan,6281MLC00000321,017-202101-03.xlsx,11,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",OJUS PETROCHEMICALS LLPC 289 NIRALA\n NAGAR LU...,MASS PVC RESIN B-57
4,NAN YA PLASTICS CORPORATION201 TUN HWA NORTH R...,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,210221TAIWAN,nan,021,nan,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...,nan,L072875,017-202101-03.xlsx,27,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)",AHMED SAEED AFIFI FACTORY CO. FORRE\nSINS LTD....,(鄰苯二甲酐)\nPHTHALIC ANHYDRIDE
5,"FORMOSA PLASTICS CORPORATION201,TUNG HWA NORTH...",+ COMMODITY: LLDPE TAISOX 3470+ QUANTITY: 32.0...,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...,210315 IN TAIWAN,210215,10/AFTER DATE OF TRANSPORT DOCUMENTS,ALL BANKING CHARGES OUTSIDE VIETNAMINCLUDING R...,+ UPON RECEIPT OF ALL DOCUMENTS SENT TO US(VIE...,MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY...,VBAAVNVX310,2100ILS210100001,017-202101-03.xlsx,18,VIET NAM BANK FOR AGRICULTURE \nAND RURAL DEVE...,MINH KHANG CHEMICAL TRADINGJOINT ST\nOCK COMPA...,LINEAR LOW DENSITY\nPOLYETHYLENE RESIN\nTAISOX...


# 去除不必要的東西

In [21]:
import re

def preprocess(x):
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 3.去除換行符號
    x = str.strip(x) # 4.移除左右空白
    return x

df['45A'] = df['45A'].apply(preprocess)
df['SPEC'] = df['SPEC'].apply(preprocess)
df.loc[:,['45A','SPEC']].head()

,45A,SPEC
3,MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,MASS PVC RESIN B-57
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2FC...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470
7,COMMODITY ...,POLYESTER PARTIALYORIENTED YARN
9,COMMODITY ...,POLYESTER FILAMENT YARN


# 寶典標註

In [22]:
import numpy as np

寶典 = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx')
產品集合 = set(寶典['品名'].values)
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:# 在寶典中搜索
            if p in df.loc[i,'45A']: 
                products.append(p)
        try:
            labels[i] = max(products,key=len) # 選最長的產品
        except:
            labels[i] = np.nan # 找不到就算了
    predict = pd.DataFrame(index=labels.keys(),columns=['Baodian_predict'])
    predict['Baodian_predict'] = labels.values()
    return predict
predict = Collection_method(df,產品集合)
df = df.join(predict)
df

<ipython-input-22-c9c8b331b91e>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


,59,45A,46A,47A,31D,44C,48,71D,78,50,51A,LCNO,from,EXPNO,LCBK,CU1,SPEC,Baodian_predict
3,"FORMOSA PLASTICS CORPORATION201,TUNG HWA N ROA...",MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,210321TAIWAN,210228,21/(REFER FIELD 47A CLAUSE NO.8),ALL BANK CHARGES OUTSIDE INDIA ARETO THE ACCOU...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...,nan,6281MLC00000321,017-202101-03.xlsx,11,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",OJUS PETROCHEMICALS LLPC 289 NIRALA\n NAGAR LU...,MASS PVC RESIN B-57,PVC RESIN B-57
4,NAN YA PLASTICS CORPORATION201 TUN HWA NORTH R...,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,210221TAIWAN,nan,021,nan,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...,nan,L072875,017-202101-03.xlsx,27,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)",AHMED SAEED AFIFI FACTORY CO. FORRE\nSINS LTD....,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE PA
5,"FORMOSA PLASTICS CORPORATION201,TUNG HWA NORTH...",COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2FC...,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...,210315 IN TAIWAN,210215,10/AFTER DATE OF TRANSPORT DOCUMENTS,ALL BANKING CHARGES OUTSIDE VIETNAMINCLUDING R...,+ UPON RECEIPT OF ALL DOCUMENTS SENT TO US(VIE...,MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY...,VBAAVNVX310,2100ILS210100001,017-202101-03.xlsx,18,VIET NAM BANK FOR AGRICULTURE \nAND RURAL DEVE...,MINH KHANG CHEMICAL TRADINGJOINT ST\nOCK COMPA...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470,LLDPE TAISOX
7,"NAN YA PLASTICS CORPORATIONNO.201,TUNG HWA N.R...",COMMODITY ...,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,210404IN THE COUNTRY OF BENEFICIARY,210320,15,ALL BANKING CHARGES OUTSIDE JAPANARE FOR BENEF...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-...",nan,LC008000033701,017-202101-03.xlsx,25,"THE MIZUHO BANK,LTD.,HEAD OFFICE","CHORI CO., LTD.TEL NO.81-76-232-300\n8FAX NO.8...",POLYESTER PARTIALYORIENTED YARN,POLYESTER PARTIALY ORIENTED YARN
9,"NAN YA PLASTICS CORPORATIONNO.201,TUNG HWA N.R...",COMMODITY ...,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,210404IN THE COUNTRY OF BENEFICIARY,210320,15,ALL BANKING CHARGES OUTSIDE JAPANARE FOR BENEF...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-...",nan,LC008000033701,017-202101-03.xlsx,25,"THE MIZUHO BANK,LTD.,HEAD OFFICE","CHORI CO., LTD.TEL NO.81-76-232-300\n8FAX NO.8...",POLYESTER FILAMENT YARN,POLYESTER PARTIALY ORIENTED YARN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20727,FORMOSA CHEMICALS AND FIBRE\r\nCORPORATION\r\n...,CFR KOBE JAPANVISCOSE RAYON STAPLE FIBER15DX40...,+ COMMERCIAL INVOICE IN 3 ORIGINAL\r\n+ 2/3 SE...,+ T.T.REIMBURSEMENT : PROHIBITED\r\n+ INSURANC...,210320AT THE NEGOTIATING BANK,210310,010,ALL BANKING CHGS INCLUDING REIMB\r\nCOMM OUTSI...,"ALL DOCS TO BE SENT DIRECTLY TO US (3-10-19, M...","MARUBENI INTEX CO.,LTD.\r\n1-2-1 DOJIMAHAMA, K...",nan,101LCS-67767362,822-202007-12.xlsx,41,SUMITOMO MITSUI BANKING \nCORP. JAPAN,"MARUBENI INTEX CO.,LTD.\n1-2-1 DOJIMAHAMA, KIT...",VISCOSE STAPLE FIBER 15DX40MM SEMI DULLNON-WOV...,VISCOSE RAYON STAPLE FIBER
20729,FORMOSA CHEMICALS AND FIBRE\r\nCORPORATION\r\n...,CFR KOBE JAPANVISCOSE RAYON STAPLE FIBER15DX40...,+ COMMERCIAL INVOICE IN 3 ORIGINAL\r\n+ 2/3 SE...,+ T.T.REIMBURSEMENT : PROHIBITED\r\n+ INSURANC...,210320AT THE NEGOTIATING BANK,210310,010,ALL BANKING CHGS INCLUDING REIMB\r\nCOMM OUTSI...,"ALL DOCS TO BE SENT DIRECTLY TO US (3-10-19, M...","MARUBENI INTEX CO.,LTD.\r\n1-2-1 DOJIMAHAMA, K...",nan,1

# 製作Y_label

In [23]:
ok_ = 0
no_ = 0
y_label = []

for i in tqdm(range(len(df))):
    # 1.判斷SPEC是否匹配
    if str(df['SPEC'].values[i]) in str(df['45A'].values[i]):
        y_label.append(df['SPEC'].values[i])
        ok_ += 1
        continue #continue：強制跳出 ❮本次❯ 迴圈，繼續進入下一圈
    
    # 2.如果SPEC無法匹配到則可以判斷寶典是否匹配
    if str(df['Baodian_predict'].values[i]) in str(df['45A'].values[i]):
        y_label.append(df['Baodian_predict'].values[i])
        ok_ += 1
        continue #continue：強制跳出 ❮本次❯ 迴圈，繼續進入下一圈
    
    # 3.如果都不匹配 則會觸發以下程序
    no_ += 1
    y_label.append(np.nan)

print(ok_,no_)
df['Y_label'] = y_label
df.head()

<ipython-input-23-393d6a0b5474>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(df))):



4212 27


,59,45A,46A,47A,31D,44C,48,71D,78,50,51A,LCNO,from,EXPNO,LCBK,CU1,SPEC,Baodian_predict,Y_label
3,"FORMOSA PLASTICS CORPORATION201,TUNG HWA N ROA...",MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,210321TAIWAN,210228,21/(REFER FIELD 47A CLAUSE NO.8),ALL BANK CHARGES OUTSIDE INDIA ARETO THE ACCOU...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...,nan,6281MLC00000321,017-202101-03.xlsx,11,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",OJUS PETROCHEMICALS LLPC 289 NIRALA\n NAGAR LU...,MASS PVC RESIN B-57,PVC RESIN B-57,MASS PVC RESIN B-57
4,NAN YA PLASTICS CORPORATION201 TUN HWA NORTH R...,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,210221TAIWAN,nan,021,nan,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...,nan,L072875,017-202101-03.xlsx,27,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)",AHMED SAEED AFIFI FACTORY CO. FORRE\nSINS LTD....,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE PA,PHTHALIC ANHYDRIDE
5,"FORMOSA PLASTICS CORPORATION201,TUNG HWA NORTH...",COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2FC...,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...,210315 IN TAIWAN,210215,10/AFTER DATE OF TRANSPORT DOCUMENTS,ALL BANKING CHARGES OUTSIDE VIETNAMINCLUDING R...,+ UPON RECEIPT OF ALL DOCUMENTS SENT TO US(VIE...,MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY...,VBAAVNVX310,2100ILS210100001,017-202101-03.xlsx,18,VIET NAM BANK FOR AGRICULTURE \nAND RURAL DEVE...,MINH KHANG CHEMICAL TRADINGJOINT ST\nOCK COMPA...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470,LLDPE TAISOX,LLDPE TAISOX
7,"NAN YA PLASTICS CORPORATIONNO.201,TUNG HWA N.R...",COMMODITY ...,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,210404IN THE COUNTRY OF BENEFICIARY,210320,15,ALL BANKING CHARGES OUTSIDE JAPANARE FOR BENEF...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-...",nan,LC008000033701,017-202101-03.xlsx,25,"THE MIZUHO BANK,LTD.,HEAD OFFICE","CHORI CO., LTD.TEL NO.81-76-232-300\n8FAX NO.8...",POLYESTER PARTIALYORIENTED YARN,POLYESTER PARTIALY ORIENTED YARN,POLYESTER PARTIALY ORIENTED YARN
9,"NAN YA PLASTICS CORPORATIONNO.201,TUNG HWA N.R...",COMMODITY ...,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...,210404IN THE COUNTRY OF BENEFICIARY,210320,15,ALL BANKING CHARGES OUTSIDE JAPANARE FOR BENEF...,ALL DOCUMENTS INCLUDING BENEFICIARY'S DRAFT(S)...,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-...",nan,LC008000033701,017-202101-03.xlsx,25,"THE MIZUHO BANK,LTD.,HEAD OFFICE","CHORI CO., LTD.TEL NO.81-76-232-300\n8FAX NO.8...",POLYESTER FILAMENT YARN,POLYESTER PARTIALY ORIENTED YARN,POLYESTER FILAMENT YARN


# 斷詞模型

In [24]:
import wordninja
from wordninja import LanguageModel
import re

寶典 = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx')
產品集合 = list(set(寶典['品名'].values))
產品集合 = [str(i).lower() for i in 產品集合]
print(產品集合[:5])

with open('../data/wordninja_words.txt',encoding="utf-8") as f:
    wordninja_words_lst = f.read().split('\n')
print(wordninja_words_lst[:5])

SPEC集合 = df['Y_label'].values.tolist()
SPEC集合 = [str(i).lower() for i in SPEC集合]
print(SPEC集合[:5])

全部集合 = list(set(產品集合+SPEC集合+wordninja_words_lst))
print(全部集合[:5])

with open('全部集合.txt', 'w',encoding="utf-8") as f:
    lines = [i + '\n' for i in 全部集合]
    f.writelines(lines)
import gzip
import shutil
with open('全部集合.txt', 'rb') as f_in, gzip.open('全部集合.txt.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

class LanguageModel2(LanguageModel):
    def split(self, s):
        _SPLIT_RE = re.compile("[^a-zA-Z0-9-']+")
        l = [self._split(x) for x in _SPLIT_RE.split(s)]
        return [item for sublist in l for item in sublist]

lm = LanguageModel2('全部集合.txt.gz')
' '.join(lm.split(df['45A'].values[0]))

['pvc copolymer', 'pvc heat stabilizer tm-2080', 'geomembrane sheets', 'orthotype vnsatvrated', 'rigid pvc sheet  pvc sheet tn-155 ']
['the', 'of', 'in', 'a', 'and']
['mass pvc resin b-57', 'phthalic anhydride', 'lldpe taisox', 'polyester partialy oriented yarn', 'polyester filament yarn']
['', 'petrovna', 'preens', 'geok', 'dappled']


'MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US DM TAS PER PROFORMA INVOICE NO A 0 B 002 - 11 D A T E D 10 - 12 - 2020 C I F PIPA VAV PORT INDIA INC O TERMS 2010'

# 斷詞處理

In [25]:
lst = []
for i in tqdm(range(len(df))):
    lst.append(' '.join(lm.split(df['45A'].values[i])))
df['45A'] = lst
df[['45A','Y_label','SPEC']].head()

<ipython-input-25-25fa5f52d0db>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(df))):


,45A,Y_label,SPEC
3,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,MASS PVC RESIN B-57
4,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470
7,COMMODITY AMOUNT POLYESTER TEXTURED YAR NHS CO...,POLYESTER PARTIALY ORIENTED YARN,POLYESTER PARTIALYORIENTED YARN
9,COMMODITY AMOUNT POLYESTER TEXTURED YAR NHS CO...,POLYESTER FILAMENT YARN,POLYESTER FILAMENT YARN


# 清洗

In [26]:
df = df.dropna(subset=['45A','Y_label'],axis=0).reset_index(drop=True)
print(df.shape)
keep_lst = []
for i in range(len(df)):
    if df.iloc[i]['Y_label'] in df.iloc[i]['45A']:
        keep_lst.append(i)
df = df.iloc[keep_lst]
print(df.shape)

(4209, 19)
(4062, 19)


In [27]:
df[['45A','Y_label']].head()

,45A,Y_label
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010


# 起始結束標註

In [28]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['45A'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df.head()

,59,45A,46A,47A,31D,44C,48,71D,78,50,...,LCNO,from,EXPNO,LCBK,CU1,SPEC,Baodian_predict,Y_label,string_Y_1,string_Y_2
0,"FORMOSA PLASTICS CORPORATION201,TUNG HWA N ROA...",MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,210321TAIWAN,210228,21/(REFER FIELD 47A CLAUSE NO.8),ALL BANK CHARGES OUTSIDE INDIA ARETO THE ACCOU...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...,...,6281MLC00000321,017-202101-03.xlsx,11,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",OJUS PETROCHEMICALS LLPC 289 NIRALA\n NAGAR LU...,MASS PVC RESIN B-57,PVC RESIN B-57,MASS PVC RESIN B-57,0,19
1,NAN YA PLASTICS CORPORATION201 TUN HWA NORTH R...,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,210221TAIWAN,nan,021,nan,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...,...,L072875,017-202101-03.xlsx,27,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)",AHMED SAEED AFIFI FACTORY CO. FORRE\nSINS LTD....,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE PA,PHTHALIC ANHYDRIDE,0,18
2,"FORMOSA PLASTICS CORPORATION201,TUNG HWA NORTH...",COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...,210315 IN TAIWAN,210215,10/AFTER DATE OF TRANSPORT DOCUMENTS,ALL BANKING CHARGES OUTSIDE VIETNAMINCLUDING R...,+ UPON RECEIPT OF ALL DOCUMENTS SENT TO US(VIE...,MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY...,...,2100ILS210100001,017-202101-03.xlsx,18,VIET NAM BANK FOR AGRICULTURE \nAND RURAL DEVE...,MINH KHANG CHEMICAL TRADINGJOINT ST\nOCK COMPA...,LINEAR LOW DENSITYPOLYETHYLENE RESINTAISOX 3470,LLDPE TAISOX,LLDPE TAISOX,10,22
5,"FORMOSA PLASTICS CORPORATION201,TUNG HWA NORTH...",ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,+1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL (S)...,+EACH SET OF DOCUMENTS PRESENTED WITH DISCREPA...,210314IN COUNTRY OF BENEFICIARY,210228,014/AFTER THE DATE OF SHIPMENT,ALL BANKING CHARGES OUTSIDE THEISSUING BANK IN...,+ FORWARD DRAFTS AND DOCUMENTS TO STANDARD CHA...,"TOPRANK CHEMICAL CO., LIMITEDLEVEL 54 HOPEWELL...",...,108013002994-L,017-202101-03.xlsx,18,STANDARD CHARTERED BANK (CHINA) \nLIMITED (HAN...,"TOPRANK CHEMICAL CO., LIMITEDLEVEL \n54 HOPEWE...",HIGH DENSITYPOLYETHYLENE RESIN,HDPE TAISOX 8010,HDPE TAISOX 8010,7,23
6,"FORMOSA PLASTICS CORPORATION201,TUNG HWA NORTH...",ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,+1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL (S)...,+EACH SET OF DOCUMENTS PRESENTED WITH DISCREPA...,210314IN COUNTRY OF BENEFICIARY,210228,014/AFTER THE DATE OF SHIPMENT,ALL BANKING CHARGES OUTSIDE THEISSUING BANK IN...,+ FORWARD DRAFTS AND DOCUMENTS TO STANDARD CHA...,"TOPRANK CHEMICAL CO., LIMITEDLEVEL 54 HOPEWELL...",...,108013002994-L,017-202101-03.xlsx,18,STANDARD CHARTERED BANK (CHINA) \nLIMITED (HAN...,"TOPRANK CHEMICAL CO., LIMITEDLEVEL \n54 HOPEWE...",ETHYLENE VINYL ACETATECOPOLYMERTAISOX 7360M,HDPE TAISOX 8010,HDPE TAISOX 8010,7,23


In [29]:
assert df['45A'].values[0][df['string_Y_1'].values[0]:df['string_Y_2'].values[0]] == df['Y_label'].values[0]

In [30]:
for i in range(10):
    print(df['45A'].values[i][df['string_Y_1'].values[i]:df['string_Y_2'].values[i]],df['Y_label'].values[i])

MASS PVC RESIN B-57 MASS PVC RESIN B-57
PHTHALIC ANHYDRIDE PHTHALIC ANHYDRIDE
LLDPE TAISOX LLDPE TAISOX
HDPE TAISOX 8010 HDPE TAISOX 8010
HDPE TAISOX 8010 HDPE TAISOX 8010
HDPE TAISOX 8010 HDPE TAISOX 8010
TAISOX TAISOX
GASOIL GASOIL
GASOIL GASOIL
EVA TAISOX EVA TAISOX


In [31]:
print(df.shape)

(4062, 21)


# 保存

In [32]:
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')